In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_train = pd.read_csv('TRAIN.csv')
df_test = pd.read_csv('TEST.csv')

In [3]:
df_train_1 = df_train.drop('id',axis=1)
df_test_1 = df_test.drop('id',axis=1)
df_train_1.head()

,text,category
0,1047 6700 4178 2912 183 6700,0
1,4035 4563 1994 154 5373 4956 6035 4393 4017 40...,1
2,5035 7685 3139 4407 254 584 6589 8518 3078,0
3,1047 2540 2578 6700 2242 230 3771 4676 5881 64...,0
4,7685 5868 237 6794 3078,0


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf = TfidfVectorizer()
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [6]:
X = df_train_1['text']
y = df_train_1['category']

In [7]:
vectorizer = CountVectorizer()
X_count = vectorizer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_count, y, test_size=0.25, random_state=1992)

In [13]:
from sklearn.metrics import precision_score
log_reg = LogisticRegression(class_weight='balanced')
log_reg.fit(X_count,y)
y_pred = log_reg.predict(X_test)
print(precision_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred))

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       626
           1       1.00      1.00      1.00        24

    accuracy                           1.00       650
   macro avg       1.00      1.00      1.00       650
weighted avg       1.00      1.00      1.00       650



In [14]:
test_count = vectorizer.transform(df_test_1['text'])
y_pred_final = log_reg.predict(test_count)
submission = pd.DataFrame({"category": y_pred_final})
submission.to_csv(('final2_submission.csv'),index=False)

In [18]:
tv = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2),sublinear_tf=True)
X_tfidf = tv.fit_transform(X)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=1992)

In [19]:
log_reg.fit(X_tfidf,y)
y_pred = log_reg.predict(X_test_tfidf)
print(precision_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred))

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       626
           1       1.00      1.00      1.00        24

    accuracy                           1.00       650
   macro avg       1.00      1.00      1.00       650
weighted avg       1.00      1.00      1.00       650



In [20]:
test_tfidf = tv.transform(df_test_1['text'])
y_pred_final = log_reg.predict(test_tfidf)
submission = pd.DataFrame({"category": y_pred_final})
submission.to_csv(('final2_tfidf_submission.csv'),index=False)

In [21]:
tv1 = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,1),sublinear_tf=True)
X_tfidf = tv1.fit_transform(X)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.25, random_state=1992)
log_reg.fit(X_tfidf,y)
y_pred = log_reg.predict(X_test_tfidf)
print(precision_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred))

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       626
           1       1.00      1.00      1.00        24

    accuracy                           1.00       650
   macro avg       1.00      1.00      1.00       650
weighted avg       1.00      1.00      1.00       650



In [22]:
test_tfidf = tv1.transform(df_test_1['text'])
y_pred_final = log_reg.predict(test_tfidf)
submission = pd.DataFrame({"category": y_pred_final})
submission.to_csv(('1_2_tfidf_submission.csv'),index=False)

In [25]:
from sklearn.model_selection import RepeatedStratifiedKFold
# define dataset
# define models and parameters
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=log_reg, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_search.fit(X_tfidf, y)
y_pred = grid_search.predict(X_test_tfidf)
print(precision_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred))

0.976665973233479
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       626
           1       0.96      0.92      0.94        24

    accuracy                           1.00       650
   macro avg       0.98      0.96      0.97       650
weighted avg       1.00      1.00      1.00       650



In [26]:
#test_tfidf = tv1.transform(df_test_1['text'])
y_pred_final = grid_search.predict(test_tfidf)
submission = pd.DataFrame({"category": y_pred_final})
submission.to_csv(('Grid_1_2_tfidf_submission.csv'),index=False)